<a href="https://colab.research.google.com/github/Akhilez/justcode/blob/master/Assignments/DL/hw8/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Deep Learning CS6073 Assignment 8

    By Akhil Kanna Devarashetti
    
Question:

    This programming assignment is based on https://github.com/pytorch/examples/tree/master/word_language_model
    But we will only run the Transformer.
    Download train.txt, valid.txt, and test.txt to ./data/wikitext-2/.
    You may need to run python main.py with specification of the selection of Transformer, or python main2.py, 
    which along with model2.py, is a simplified version only for the Transformer and with a few epochs.
    We need data.py to start
    and generate.py to show the learning result.
    Show that you indeed have spent time in studying and running the programs.

In [0]:
import math
import torch
import torch.nn as nn
import os
from io import open
import torch.nn.functional as F

In [5]:
!apt-get install tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 25 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (203 kB/s)
Selecting previously unselected package tree.
(Reading database ... 145118 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [0]:
bptt = 4
loginterval = 200

In [0]:
!mkdir -p data/wikitext-2
!curl https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/test.txt > data/wikitext-2/test.txt
!curl https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/train.txt > data/wikitext-2/train.txt
!curl https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/valid.txt > data/wikitext-2/valid.txt
!ls data/wikitext-2/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1227k  100 1227k    0     0  2673k      0 --:--:-- --:--:-- --:--:-- 2667k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.2M  100 10.2M    0     0  19.8M      0 --:--:-- --:--:-- --:--:-- 19.7M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1095k  100 1095k    0     0  2090k      0 --:--:-- --:--:-- --:--:-- 2086k
test.txt  train.txt  valid.txt


## Lets take a small dataset to analyse what's happening.

In [7]:
!mkdir -p data/wikitext-2
!echo -e 'a quick brown fox jumps over the lazy dog\na lazy dog is not a brown fox' > data/wikitext-2/test.txt
!echo -e "the quick brown fox moves faster than the lazy dog\nthis is because the quick brown fox is smarter than the lazy dog" > data/wikitext-2/train.txt
!echo -e "a quick brown fox jumps over the lazy dog\na lazy dog is not a brown fox" > data/wikitext-2/valid.txt
!tree data

data
└── wikitext-2
    ├── test.txt
    ├── train.txt
    └── valid.txt

1 directory, 3 files


In [0]:
!cat data/wikitext-2/train.txt

the quick brown fox moves faster than the lazy dog
this is because the quick brown fox is smarter than the lazy dog


## The dictionary class saves the mapping between the words and their indices.
## Every word is assigned an integer based on the order of its occurance.

    Example input: "a quick brown fox jumps over the lazy dog\na lazy dog is not a brown fox <eos>"

    Output: 
    {'a': 0, 'quick': 1, 'brown': 2, 'fox': 3, 'jumps': 4, 'over': 5, 'the': 6, 'lazy': 7, 'dog': 8, 'is': 9, 'not': 10, '<eos>': 11}

In [0]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)

In [9]:
sample_input = "a quick brown fox jumps over the lazy dog\na lazy dog is not a brown fox <eos>"

sample_dict = Dictionary()

for word in sample_input.split():
    sample_dict.add_word(word)

print(sample_dict.word2idx)

{'a': 0, 'quick': 1, 'brown': 2, 'fox': 3, 'jumps': 4, 'over': 5, 'the': 6, 'lazy': 7, 'dog': 8, 'is': 9, 'not': 10, '<eos>': 11}


Corpus class will create the dictionary for all three datasets.

    For example:
    test data = "a quick brown fox jumps over the lazy dog a lazy dog is not a brown fox"
    corpus.test = tensor([14,  1,  2,  3, 15, 16,  0,  7, 17,  7, 12,  9, 18, 14,  2,  3, 13])

Note that 'a' is repeated twice and we see it's id 14 repeated twice too.

In [0]:
class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))

    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r', encoding="utf8") as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r', encoding="utf8") as f:
            idss = []
            for line in f:
                words = line.split() + ['<eos>']
                ids = []
                for word in words:
                    ids.append(self.dictionary.word2idx[word])
                idss.append(torch.tensor(ids).type(torch.int64))
            ids = torch.cat(idss)

        return ids

In [0]:
batch_size = 20
eval_batch_size = 10

corpus = Corpus('./data/wikitext-2')

In [12]:
print(f'Dictionary: {corpus.dictionary.word2idx}')
print('\nTest data:')
!cat data/wikitext-2/test.txt
print(f'\ncorpus.test: {corpus.test}')

Dictionary: {'the': 0, 'quick': 1, 'brown': 2, 'fox': 3, 'moves': 4, 'faster': 5, 'than': 6, 'lazy': 7, 'dog': 8, '<eos>': 9, 'this': 10, 'is': 11, 'because': 12, 'smarter': 13, 'a': 14, 'jumps': 15, 'over': 16, 'not': 17}

Test data:
a quick brown fox jumps over the lazy dog
a lazy dog is not a brown fox

corpus.test: tensor([14,  1,  2,  3, 15, 16,  0,  7,  8,  9, 14,  7,  8, 11, 17, 14,  2,  3,
         9])


As given in the documentation, the batchify function will take the complete dataset as input string and converts it into batches.

    


In [0]:
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data

In [14]:
print(corpus.test)
sample_batches = batchify(corpus.test, 3)
print(sample_batches)

tensor([14,  1,  2,  3, 15, 16,  0,  7,  8,  9, 14,  7,  8, 11, 17, 14,  2,  3,
         9])
tensor([[14,  0,  8],
        [ 1,  7, 11],
        [ 2,  8, 17],
        [ 3,  9, 14],
        [15, 14,  2],
        [16,  7,  3]])


get_batch will actually create a batch of sequence size bptt from index i. Source is the output from batchify()

In [0]:
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i + seq_len]
    target = source[i + 1:i + 1 + seq_len].view(-1)
    return data, target

In [25]:
print(get_batch(sample_batches, 0))
print(get_batch(sample_batches, 1))

(tensor([[14,  0,  8],
        [ 1,  7, 11],
        [ 2,  8, 17],
        [ 3,  9, 14]]), tensor([ 1,  7, 11,  2,  8, 17,  3,  9, 14, 15, 14,  2]))
(tensor([[ 1,  7, 11],
        [ 2,  8, 17],
        [ 3,  9, 14],
        [15, 14,  2]]), tensor([ 2,  8, 17,  3,  9, 14, 15, 14,  2, 16,  7,  3]))


In [0]:
train_data = batchify(corpus.train, batch_size)
val_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, eval_batch_size)

In [0]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):

        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


class TransformerModel(nn.Module):
    """Container module with an encoder, a recurrent or transformer module, and a decoder."""

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, has_mask=True):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return F.log_softmax(output, dim=-1)


In [0]:
emsize = 100
nhead = 2
nhid = 64
nlayers = 2
dropout = 0.2
ntokens = len(corpus.dictionary)

model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout)
criterion = nn.NLLLoss()

In [0]:
def train():
    model.train()
    total_loss = 0.
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        model.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()

        total_loss += loss.item()

        if batch % loginterval == 0 and batch > 0:
            cur_loss = total_loss / loginterval
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // bptt, lr,
                cur_loss, math.exp(cur_loss)))
            total_loss = 0


In [0]:
def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output = model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    print(data_source)
    return total_loss / (len(data_source) - 1)

In [34]:
# Loop over epochs.
lr = 20
best_val_loss = None
epochs = 5
# At any point you can hit Ctrl + C to break out of training early.
try:
    for epoch in range(1, epochs + 1):
        train()
        val_loss = evaluate(val_data)
        print('-' * 89)
        print('| end of epoch {:3d} | valid loss {:5.2f} | '
              'valid ppl {:8.2f}'.format(epoch, val_loss, math.exp(val_loss)))
        print('-' * 89)
        # Save the model if the validation loss is the best we've seen so far.
        if not best_val_loss or val_loss < best_val_loss:
            with open('model.pt', 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss
        else:
            # Anneal the learning rate if no improvement has been seen in the validation dataset.
            lr /= 4.0
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

tensor([[14,  1,  2,  3, 15, 16,  0,  7,  8,  9]])


ZeroDivisionError: ignored

In [0]:
test_loss = evaluate(test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

| End of training | test loss 10.58 | test ppl 39508.86
